# Ride Booking Analysis: Customer Satisfaction & Operational Insights

**Analyst:** [Your Name]  
**Date:** December 2025  
**Dataset:** 150,000 ride bookings

---

## Executive Summary

This analysis examines ride booking data to identify key drivers of customer satisfaction and operational inefficiencies.

**Key Findings:**
- 38% of rides fail due to cancellations or operational issues, representing **₹29M+ in potential lost revenue**
- Pickup time under 3 minutes yields highest satisfaction (4.236/5)
- 21.9% of customer cancellations involve drivers asking customers to cancel (system gaming)
- 2% mechanical breakdown rate causing incomplete rides
- Vehicle type has minimal impact on ratings (all 4.23-4.24/5)

**Revenue Impact:** ₹47.26M realized from completed rides.

---
## 1. Data Loading & Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from rides_pipeline import RideBookings

pd.set_option('display.max_columns', None)

ride = RideBookings()
ride.load_pipeline()
df = ride.df

---
## 2. Dataset Overview

In [ ]:
print(f"Dataset Shape: {df.shape}")
print(f"Total Records: {df.shape[0]:,}")
print(f"Total Features: {df.shape[1]}")
print("\nColumns:", df.columns.tolist())

In [ ]:
df.describe()

### Missing Data Patterns
- 48,000 rides (32%) missing financial data - likely cancelled/incomplete
- 139,500 rides (93%) missing customer cancellation data
- 123,000 rides (82%) missing driver cancellation data
- 57,000 rides (38%) missing ratings

---
## 3. Booking Status Analysis

In [ ]:
booking_status = df['Booking Status'].value_counts()
print("Booking Status Distribution:")
print(booking_status)
print(f"\nCompletion Rate: {(booking_status['Completed'] / len(df)) * 100:.1f}%")
print(f"Failure Rate: {((len(df) - booking_status['Completed']) / len(df)) * 100:.1f}%")

In [ ]:
plt.figure(figsize=(10, 6))
colors = ['#2ecc71', '#e74c3c', '#e67e22', '#f39c12', '#95a5a6']
booking_status.plot(kind='bar', color=colors)
plt.title('Distribution of Booking Status', fontsize=14, fontweight='bold')
plt.xlabel('Booking Status')
plt.ylabel('Number of Bookings')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Key Insight: 38% Ride Failure Rate

- **Completed:** 93,000 (62%)
- **Cancelled by Driver:** 27,000 (18%)
- **No Driver Found:** 10,500 (7%)
- **Cancelled by Customer:** 10,500 (7%)
- **Incomplete:** 9,000 (6%)

---
## 4. Customer Satisfaction Metrics

In [ ]:
new_df = df[(df['Driver Ratings'].notna()) | (df['Customer Rating'].notna())]
bookings_df = new_df.copy()

print(f"Rides with ratings: {len(bookings_df):,}")
print(f"Average Driver Rating: {bookings_df['Driver Ratings'].mean():.2f}/5.00")
print(f"Average Customer Rating: {bookings_df['Customer Rating'].mean():.2f}/5.00")

In [ ]:
# Impute missing values
bookings_df['Avg VTAT'] = bookings_df.groupby('Vehicle Type')['Avg VTAT'].transform(lambda x: x.fillna(x.median()))
bookings_df['Avg CTAT'] = bookings_df.groupby('Vehicle Type')['Avg CTAT'].transform(lambda x: x.fillna(x.median()))
bookings_df['Hour'] = pd.to_datetime(bookings_df['Time'], format="%H:%M:%S", errors='coerce').dt.hour
bookings_df['Booking Value'] = bookings_df.groupby(['Vehicle Type','Hour'])['Booking Value'].transform(lambda x: x.fillna(x.median()))
bookings_df['Ride Distance'] = bookings_df.groupby(['Vehicle Type','Hour'])['Ride Distance'].transform(lambda x: x.fillna(x.median()))
print("Data preparation complete.")

In [ ]:
avg_ratings_vehicle = bookings_df.groupby('Vehicle Type')['Driver Ratings'].mean().sort_values(ascending=False)
print("Average Driver Rating by Vehicle Type:")
print(avg_ratings_vehicle)

plt.figure(figsize=(10, 6))
avg_ratings_vehicle.plot(kind='barh', color='skyblue')
plt.title('Average Driver Rating by Vehicle Type', fontsize=14, fontweight='bold')
plt.xlabel('Average Rating')
plt.ylabel('Vehicle Type')
plt.xlim(4.20, 4.25)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Key Insight: Vehicle Type Has Minimal Impact
All vehicle types cluster around 4.23-4.24, suggesting driver behavior matters more than vehicle type.

---
## 5. Pickup Time Impact on Ratings

In [ ]:
bins = np.linspace(0, bookings_df['Avg VTAT'].max(), 6)
bookings_df['VTAT_bin'] = pd.cut(bookings_df['Avg VTAT'], bins)
avg_rating_per_bin = bookings_df.groupby('VTAT_bin')['Driver Ratings'].mean()
print("Average Rating by Pickup Time:")
print(avg_rating_per_bin)

In [ ]:
plt.figure(figsize=(12, 6))
avg_rating_per_bin.plot(marker='o', color='blue', linewidth=2, markersize=8)
plt.xlabel('Average Pickup Time (minutes)', fontsize=12)
plt.ylabel('Average Driver Rating', fontsize=12)
plt.title('Average Driver Rating vs Pickup Time (Binned)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Critical Finding: U-Shaped Pattern

1. **0-3 minutes:** Highest (4.236) - exceeds expectations
2. **6-9 minutes:** Lowest (4.229) - frustration zone
3. **9-12 minutes:** Recovery (4.234) - adjusted expectations
4. **12-15 minutes:** Drop (4.226) - patience exhausted

**Strategy:** Target sub-3 minute pickups in high-density areas.

---
## 6. Trip Duration Impact

In [ ]:
bins = [0, 5, 10, 20, 30, 45]
labels = ['0–5', '5–10', '10–20', '20–30', '30–45']
bookings_df['CTAT_bin'] = pd.cut(bookings_df['Avg CTAT'], bins=bins, labels=labels, include_lowest=True)
avg_rating_ctat = bookings_df.groupby('CTAT_bin')['Driver Ratings'].mean()
print("Average Rating by Trip Duration:")
print(avg_rating_ctat)
print("\nSample Size per Bin:")
print(bookings_df['CTAT_bin'].value_counts().sort_index())

In [ ]:
plt.figure(figsize=(10, 6))
avg_rating_ctat.plot(marker='o', color='green', linewidth=2, markersize=8)
plt.title("Average Driver Rating vs Trip Duration (CTAT)", fontsize=14, fontweight='bold')
plt.xlabel("Trip Duration Range (minutes)", fontsize=12)
plt.ylabel("Average Driver Rating", fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Key Insight: Trip Duration Doesn't Matter
Variation is only 0.0002 points. Customers judge rides on pickup experience and driver behavior, not trip length.

---
## 7. Customer Cancellation Analysis

In [ ]:
cancellation_reasons_customer = df[df['Reason for cancelling by Customer'].notna()]
reasons_customer = cancellation_reasons_customer['Reason for cancelling by Customer'].value_counts()
print("Customer Cancellation Reasons:")
print(reasons_customer)
print(f"\nTotal: {len(cancellation_reasons_customer):,}")

In [ ]:
plt.figure(figsize=(12, 6))
reasons_customer.plot(kind='barh', color='coral')
plt.title('Customer Cancellation Reasons', fontsize=14, fontweight='bold')
plt.xlabel('Number of Cancellations')
plt.ylabel('Reason')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
cancellation_reasons_customer['Hour'] = pd.to_datetime(cancellation_reasons_customer['Time'], format="%H:%M:%S", errors='coerce').dt.hour
drivers_asked_to_cancel = cancellation_reasons_customer[cancellation_reasons_customer['Reason for cancelling by Customer'] == 'Driver asked to cancel']
hp_df = drivers_asked_to_cancel['Hour'].value_counts().sort_index().to_frame()

plt.figure(figsize=(14, 4))
sns.heatmap(hp_df.T, annot=True, fmt='d', cmap='RdYlGn', cbar_kws={'label': 'Cancellations'})
plt.title("'Driver Asked to Cancel' - Hourly Heatmap", fontsize=14, fontweight='bold')
plt.ylabel("")
plt.xlabel("Hour of Day")
plt.tight_layout()
plt.show()

### Critical Issue: Driver Gaming (21.9% of cancellations)

**Peak Hours:**
- 17-18h: 195-166 cancellations (cherry-picking profitable rides)
- 20-23h: 147-130 (avoiding shift penalties)
- 0-5h: 26-52 (unprofitable rides)

**Actions:** Detection algorithms, equal penalties, peak monitoring, customer education.

---
## 8. Driver Cancellation Patterns

In [ ]:
cancellation_reasons_driver = df[df['Driver Cancellation Reason'].notna()]
reasons_driver = cancellation_reasons_driver['Driver Cancellation Reason'].value_counts()
print("Driver Cancellation Reasons:")
print(reasons_driver)
print(f"\nTotal: {len(cancellation_reasons_driver):,}")

In [ ]:
plt.figure(figsize=(12, 6))
reasons_driver.plot(kind='barh', color='salmon')
plt.title('Driver Cancellation Reasons', fontsize=14, fontweight='bold')
plt.xlabel('Number of Cancellations')
plt.ylabel('Reason')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Analysis
Even distribution (~25% each): Customer-related (25.3%), Sick customer (25%), Car issues (24.9%), Excess passengers (24.8%)

---
## 9. Vehicle Type Analysis

In [ ]:
reasons_for_cancelling_per_vehicle = cancellation_reasons_customer[['Vehicle Type','Reason for cancelling by Customer']].value_counts()
df_plot = reasons_for_cancelling_per_vehicle.reset_index(name='Count').pivot(index='Vehicle Type', columns='Reason for cancelling by Customer', values='Count').fillna(0)
print("Cancellations by Vehicle Type:")
print(df_plot)

In [ ]:
plt.figure(figsize=(14, 7))
df_plot.plot(kind='bar', figsize=(14, 7))
plt.xlabel("Vehicle Type", fontsize=12)
plt.ylabel("Number of Cancellations", fontsize=12)
plt.title("Cancellation Reasons per Vehicle Type", fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Key Findings
- **Auto:** 3,400 cancellations (impulsive bookings)
- **Go Mini:** 460 AC complaints (critical issue)
- **Premium Sedan:** 260 AC issues (unacceptable)

---
## 10. Incomplete Rides Analysis

In [ ]:
incomplete_rides = df[df['Incomplete Rides'].notna()]
reasons = incomplete_rides['Incomplete Rides Reason'].value_counts()
print("Incomplete Ride Reasons:")
print(reasons)
print(f"\nTotal: {len(incomplete_rides):,} ({(len(incomplete_rides)/len(df))*100:.2f}%)")

In [ ]:
plt.figure(figsize=(10, 6))
reasons.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Incomplete Rides - Reason Distribution', fontsize=14, fontweight='bold')
plt.ylabel('')
plt.tight_layout()
plt.show()

### Critical Issue: 2% Vehicle Breakdown Rate

**9,000 incomplete rides:**
- Customer Demand: 3,040 (33.8%)
- Vehicle Breakdown: 3,012 (33.5%) - **4x industry standard**
- Other Issue: 2,948 (32.7%)

**Actions:** Fleet audit, predictive maintenance, better categorization.

---
## 11. Revenue Analysis

In [ ]:
completed_rides = df[(df['Incomplete Rides'].isna()) & (df['Booking Value'].notna())]
total_revenue = completed_rides['Booking Value'].sum()

print(f"Total Revenue: ₹{total_revenue:,.0f}")
print(f"Completed Rides: {len(completed_rides):,}")
print(f"Average Booking Value: ₹{completed_rides['Booking Value'].mean():.2f}")

In [ ]:
completion_rate = len(completed_rides) / len(df)
failure_rate = 1 - completion_rate
avg_booking_value = completed_rides['Booking Value'].mean()
failed_bookings = len(df) - len(completed_rides)
potential_lost_revenue = failed_bookings * avg_booking_value

print(f"\n--- Revenue Opportunity ---")
print(f"Completion Rate: {completion_rate:.1%}")
print(f"Failure Rate: {failure_rate:.1%}")
print(f"Failed Bookings: {failed_bookings:,}")
print(f"Potential Lost Revenue: ₹{potential_lost_revenue:,.0f}")
print(f"\nIf failure rate reduced to 20%:")
print(f"Revenue Recovery: ₹{(failed_bookings * 0.47 * avg_booking_value):,.0f}")

In [ ]:
revenue_data = {'Realized Revenue': total_revenue, 'Potential Lost Revenue': potential_lost_revenue}

plt.figure(figsize=(10, 6))
plt.bar(revenue_data.keys(), revenue_data.values(), color=['#2ecc71', '#e74c3c'])
plt.title('Revenue: Realized vs Potential Lost', fontsize=14, fontweight='bold')
plt.ylabel('Revenue (₹)', fontsize=12)
for i, (k, v) in enumerate(revenue_data.items()):
    plt.text(i, v, f'₹{v/1e6:.1f}M', ha='center', va='bottom', fontsize=11, fontweight='bold')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

### Revenue Recovery Opportunity
- Realized: ₹47.26M
- Potential Lost: ₹29M+
- If failure rate reduced 38%→20%: **₹13.6M recovery**
- Each 1% improvement = ₹753K

---
## 12. Conclusions & Recommendations

### Key Findings Summary

**1. Operational Crisis**
- 38% ride failure rate = ₹29M+ lost revenue
- 2% breakdown rate (4x industry standard)
- 21.9% gaming (drivers ask customers to cancel)

**2. Satisfaction Insights**
- Overall 4.23/5 (improvable)
- Pickup time > trip duration for satisfaction
- Sub-3 min pickups = highest rating (4.236)
- Vehicle type minimal impact (4.23-4.24)

**3. Critical Time Windows**
- Peak gaming: 5-7 PM (cherry-picking)
- Late evening: 8-11 PM (shift avoidance)
- Early morning: 12-5 AM (low demand)

---

### Strategic Recommendations

#### Immediate (0-30 Days)
1. **Combat Driver Gaming**
   - Detection algorithms
   - Equal penalties for driver-requested cancellations
   - Peak hour monitoring (5-7 PM)

2. **Fleet Maintenance Blitz**
   - Go Mini AC inspection (460 complaints)
   - Zero-tolerance AC policy for premium
   - Target: 2% → <0.5% breakdown rate

3. **Data Quality**
   - Better categorization of "Other Issue"
   - Investigate missing <10 min rides

#### Short-term (1-3 Months)
1. **Optimize Pickup Experience**
   - Incentives for sub-3 min pickups
   - Proactive 6-9 min ETA communication
   - Real-time accuracy improvements

2. **Auto Strategy**
   - Shorter cancellation windows
   - Small cancellation fees
   - Deposit for repeat cancellers

3. **Incomplete Rides Protocol**
   - Partial payment collection
   - Compensation strategy
   - Better incident tracking

#### Long-term (3-6 Months)
1. **Revenue Recovery: 38% → 20% (₹13.6M)**
   - Driver cancellations: 18% → 10%
   - No Driver Found: 7% → 3%
   - Incomplete rides: 6% → 3%

2. **Predictive Maintenance**
   - IoT vehicle monitoring
   - Breakdown prediction
   - Mandatory pre-shift checks

3. **Premium Strategy Reevaluation**
   - Focus on comfort messaging (not service)
   - Zero AC issues guarantee
   - Priority matching peak hours

4. **Driver Quality Program**
   - Behavior training over vehicle upgrades
   - Pickup speed training
   - Communication best practices

---

### KPIs to Track

| Metric | Current | Target (6mo) | Impact |
|--------|---------|--------------|--------|
| Ride Completion Rate | 62% | 80% | +₹13.6M |
| Sub-3 Min Pickup Rate | Unknown | 25% | +0.01 rating |
| Breakdown Rate | 2.0% | 0.5% | -1.5% incomplete |
| Driver Gaming | 2,295/yr | 500/yr | -78% abuse |
| Go Mini AC Complaints | 460 | 50 | -89% issues |
| Driver Cancellation | 18% | 10% | +₹6M |
| Average Rating | 4.23 | 4.35 | +2.8% satisfaction |

---

### Bottom Line

Solid foundation (4.23 ratings, ₹47.26M revenue) but **38% failure rate = enormous opportunity**.

**Biggest opportunities are operational:**
- Fix vehicle reliability (2% breakdown)
- Combat system gaming (21.9% cancellations)
- Focus on first 3 minutes
- Driver behavior > Vehicle type

**Expected Impact:** ₹13.6M+ annual recovery + 2.8% satisfaction improvement (4.23 → 4.35).

**Path to growth: Operational excellence drives both satisfaction and revenue.**

---
## Appendix: Technical Details

### Methodology
- Filtered to 93,000 rides with ratings
- Median imputation by Vehicle Type for VTAT/CTAT
- Median imputation by Vehicle Type + Hour for Booking Value/Distance
- Binned analysis for continuous variables
- Time-series heatmap for hourly patterns

### Tools
- Python 3.x, pandas, numpy, matplotlib, seaborn
- Custom RideBookings pipeline

### Data Quality Notes
**Strengths:** 150K bookings, 21 features, good temporal coverage

**Limitations:** No rides <10 min, even driver cancellation distribution, 33% "Other Issue"

### Contact
**Analyst:** [Your Name]  
**Email:** [your.email@example.com]  
**LinkedIn:** [your-linkedin]  
**GitHub:** [your-github]

---

*Portfolio project demonstrating: EDA, Business Intelligence, Data Visualization, Statistical Analysis, Strategic Recommendations*